In [1]:
from DrissionPage import ChromiumPage, ChromiumOptions
import time
import pandas as pd
import random

In [2]:
def extract_paper_details(result: ChromiumPage) -> dict:
    """
    Extracts details from a single arXiv paper result.
    Args:
        result (DrissionPage): The DrissionPage object representing the paper result.
    Returns:    
        dict: A dictionary containing the extracted details.
    """
    
    try:
        # Extract arXiv paper link
        link_element = result.ele('css:a[href*="abs"]')
        link = link_element.link if link_element else None

        # Extract pdf link
        pdf_link_element = result.ele('css:a[href*="pdf"]')
        pdf_link = pdf_link_element.link if pdf_link_element else None

        # Extract title
        title_element = result.ele('css:p.title')
        title = title_element.text.strip() if title_element else None

        # Extract authors
        authors_element = result.ele('css:p.authors')
        authors = authors_element.text.replace("Authors:", "").strip() if authors_element else None

        # Extract abstract
        abstract_element = result.ele('css:p.abstract span.abstract-full')
        abstract = abstract_element.text.strip() if abstract_element else None
        if abstract and "△ Less" in abstract:
            abstract = abstract.split("△ Less")[0].strip()

        # Extract submitted date
        submitted_element = result.ele('css:p.is-size-7')
        submitted = submitted_element.text.strip() if submitted_element else None

    except Exception as e:
        print(f"Error extracting details: {e}")

    return {
        'link': link,
        'pdf_link': pdf_link,
        'title': title,
        'authors': authors,
        'abstract': abstract,
        'submitted': submitted
    }

In [3]:
def paper_crawl(keywords: str) -> list:
    """
    Crawls arXiv for papers based on the given keywords.
    Args:
        keywords (str): The keywords to search for.     
    Returns:
        list: A list of dictionaries containing paper details.
    """

    #List of paper data
    papers_data = []

    # Navigate to the search page
    # Construct the search URL
    search_url = f'https://arxiv.org/search/?query={keywords}&searchtype=all&source=header&abstracts=show'
    print(search_url)
    
    option = ChromiumOptions()
    # Uncomment the following lines to run in headless mode or with specific options
    option.headless(on_off=True)  # Run in headless mode
    # Initialize the page
    page = ChromiumPage(addr_or_opts=option)
    try:
        page.get(search_url)
        cnt = 1
        while True:
            # Wait for the page to load fully
            page.scroll.to_bottom()
            time.sleep(random.randint(2,3))
            
            print(f"Page {cnt} loaded, extracting data...")

            # Target the result element
            arxiv_results = page.eles ('css:li.arxiv-result')

            # Loop through each result and extract details
            for result in arxiv_results:
                paper_data = {}
                try:
                    paper_data = extract_paper_details(result)
                    papers_data.append(paper_data)
                except Exception as e:
                    print(f"Error extracting data from a result: {e}")

            # Check if the "Load More" button is present
            next_button = page.ele('css:a.pagination-next')
            if next_button and 'is-invisible' not in next_button.attr('class'):
                next_button.click()
                cnt += 1
                print(f"Loading next page...")
            else:
                print("No more pages to load.")
                break

    except Exception as e:
        print(f"Error: {e}")
    
    finally:
        page.quit()
        return papers_data

In [4]:
def save_to_csv(data: list, filename: str) -> None:
    """
    Save the extracted data to a CSV file.
    """
    try:
        df = pd.DataFrame(data)
        df.drop_duplicates(subset=['link'], inplace=True)
        df.to_csv(f'arxiv/{filename}', index=False)
        print(f"Data saved to {filename}")
    except Exception as e:
        print(f"Error saving data to CSV: {e}")

In [5]:
import itertools

def combination_keywords(sets):
    """Tạo danh sách các chuỗi từ khóa từ một danh sách các bộ từ khóa."""
    if not sets:
        return []
    combinations = itertools.product(*sets)
    return [' AND '.join(combo) for combo in combinations]

def generate_all_combinations(t2sql, security, llm):
    """Tạo danh sách tất cả các tổ hợp từ khóa theo các trường hợp yêu cầu."""
    # Định nghĩa các trường hợp cần tạo tổ hợp
    cases = [
        [t2sql],                    # Chỉ t2sql
        [t2sql, security],          # t2sql + security
        [t2sql, llm],               # t2sql + llm
        [t2sql, security, llm]      # t2sql + security + llm
    ]
    
    # Tạo và hợp nhất tất cả các tổ hợp
    all_combinations = []
    for case in cases:
        all_combinations.extend(combination_keywords(case))
    
    return all_combinations

In [6]:
def crawl_arxiv():
    """
    Main function to run the pipeline.
    """
    
    t2sql = ['"text-to-sql"', '"nl2sql"', '"t2sql"', '"text2sql"', '"natural language to sql"', 
             '"semantic parsing to sql"', '"nl to sql"']
    security = ['"security"', '"access control"', '"injection"', '"prompt injection"', '"defense"', '"attack"', '"vulnerability"']
    llm = ['"llm"', '"large language model"']

    keywords = generate_all_combinations(t2sql, security, llm)

    tmp = []

    # Crawl papers
    for keyword in keywords:
        print(f"Searching for papers related to: {keyword}")
        keyword_1 = keyword.strip()
        keyword_1 = keyword_1.replace(" ", "+")
        papers_data = paper_crawl(keyword_1)
        if len(papers_data) > 0:
            keyword = keyword.replace('"', '')
            keyword = keyword.replace(' ', '_')
            save_to_csv(papers_data,f"crawl_by_{keyword}.csv")
        tmp.extend(papers_data)

    if tmp:
        # Save data to CSV
        filename = "all_arxiv_papers"
        save_to_csv(tmp, f"{filename}.csv")
        print(f"Extracted {len(tmp)} papers.")

In [7]:
crawl_arxiv()

Searching for papers related to: "text-to-sql"
https://arxiv.org/search/?query="text-to-sql"&searchtype=all&source=header&abstracts=show
Page 1 loaded, extracting data...
Loading next page...
Page 2 loaded, extracting data...
Loading next page...
Page 3 loaded, extracting data...
Loading next page...
Page 4 loaded, extracting data...
Loading next page...
Page 5 loaded, extracting data...
Loading next page...
Page 6 loaded, extracting data...
Loading next page...
Page 7 loaded, extracting data...
Loading next page...
Page 8 loaded, extracting data...
No more pages to load.
Data saved to crawl_by_text-to-sql.csv
Searching for papers related to: "nl2sql"
https://arxiv.org/search/?query="nl2sql"&searchtype=all&source=header&abstracts=show
Page 1 loaded, extracting data...
No more pages to load.
Data saved to crawl_by_nl2sql.csv
Searching for papers related to: "t2sql"
https://arxiv.org/search/?query="t2sql"&searchtype=all&source=header&abstracts=show
Page 1 loaded, extracting data...
No mo

In [8]:
import pandas as pd
from DrissionPage import ChromiumPage, ChromiumOptions

# Load the CSV file
df = pd.read_csv('arxiv/all_arxiv_papers.csv')
df.head()

dois = []

option = ChromiumOptions()
# Uncomment the following lines to run in headless mode or with specific options
option.headless(on_off=True)  # Run in headless mode
# Initialize the page
page = ChromiumPage(addr_or_opts=option)

for link in df['link']:
    page.get(link)
    doi = page.ele('css:#arxiv-doi-link').attr('href')
    print(doi)
    dois.append(doi)

df['doi'] = dois
df.to_csv('arxiv/all_arxiv_papers.csv', index=False)

https://doi.org/10.48550/arXiv.2506.07423
https://doi.org/10.48550/arXiv.2506.07245
https://doi.org/10.48550/arXiv.2506.06093
https://doi.org/10.48550/arXiv.2506.04494
https://doi.org/10.48550/arXiv.2506.03598
https://doi.org/10.48550/arXiv.2506.01710
https://doi.org/10.48550/arXiv.2506.01571
https://doi.org/10.48550/arXiv.2506.01273
https://doi.org/10.48550/arXiv.2506.00986
https://doi.org/10.48550/arXiv.2506.00912
https://doi.org/10.48550/arXiv.2506.00842
https://doi.org/10.48550/arXiv.2506.00391
https://doi.org/10.48550/arXiv.2505.23838
https://doi.org/10.48550/arXiv.2505.23804
https://doi.org/10.48550/arXiv.2505.22096
https://doi.org/10.48550/arXiv.2505.20321
https://doi.org/10.48550/arXiv.2505.20315
https://doi.org/10.48550/arXiv.2505.19988
https://doi.org/10.48550/arXiv.2505.19956
https://doi.org/10.48550/arXiv.2505.19197
https://doi.org/10.48550/arXiv.2505.18929
https://doi.org/10.48550/arXiv.2505.18744
https://doi.org/10.48550/arXiv.2505.18363
https://doi.org/10.48550/arXiv.250

In [13]:
import pandas as pd
papers = pd.read_csv("../raw_crawl_papers/arxiv/all_arxiv_papers.csv")
papers.head()

,Unnamed: 0,link,pdf_link,title,authors,abstract,submitted,doi
0,0,https://arxiv.org/abs/2506.07423,https://arxiv.org/pdf/2506.07423,SEED: Enhancing Text-to-SQL Performance and Pr...,"Janghyeon Yun, Sang-goo Lee",Text-to-SQL enables non-experts to retrieve da...,2025-06-09,https://doi.org/10.48550/arXiv.2506.07423
1,1,https://arxiv.org/abs/2506.07245,https://arxiv.org/pdf/2506.07245,SDE-SQL: Enhancing Text-to-SQL Generation in L...,"Wenxuan Xie, Yaxun Dai, Wenhao Jiang",Recent advancements in large language models (...,2025-06-08,https://doi.org/10.48550/arXiv.2506.07245
2,2,https://arxiv.org/abs/2506.06093,https://arxiv.org/pdf/2506.06093,Reinforcing Code Generation: Improving Text-to...,"Atharv Kulkarni, Vivek Srikumar","In this work, we study the problem of code gen...",2025-06-06,https://doi.org/10.48550/arXiv.2506.06093
3,3,https://arxiv.org/abs/2506.04494,https://arxiv.org/pdf/2506.04494,SQLens: An End-to-End Framework for Error Dete...,"Yue Gong, Chuan Lei, Xiao Qin, Kapil Vaidya, B...",Text-to-SQL systems translate natural language...,2025-06-04,https://doi.org/10.48550/arXiv.2506.04494
4,4,https://arxiv.org/abs/2506.03598,https://arxiv.org/pdf/2506.03598,Auto prompt sql: a resource-efficient architec...,"Zetong Tang, Qian Ma, Di Wu",Using the best Text-to-SQL methods in resource...,2025-06-04,https://doi.org/10.48550/arXiv.2506.03598


In [ ]:
# import re
# from datetime import datetime

# def extract_date(text):
#     # Tìm mẫu "Submitted DD Month, YYYY"
#     pattern = r'Submitted (\d{1,2}) ([A-Za-z]+), (\d{4})'
#     match = re.search(pattern, text)
    
#     if match:
#         day, month, year = match.groups()
#         # Chuyển đổi thành đối tượng datetime
#         try:
#             return pd.to_datetime(f"{day} {month} {year}")
#         except:
#             return None
#     return None

# # Áp dụng hàm chuyển đổi cho cột submitted
# papers['submitted'] = papers['submitted'].apply(extract_date)

In [12]:
papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  463 non-null    int64 
 1   link        463 non-null    object
 2   pdf_link    459 non-null    object
 3   title       463 non-null    object
 4   authors     463 non-null    object
 5   abstract    463 non-null    object
 6   submitted   0 non-null      object
 7   doi         463 non-null    object
dtypes: int64(1), object(7)
memory usage: 29.1+ KB


In [9]:
papers.to_csv("../raw_crawl_papers/arxiv/all_arxiv_papers.csv")